In [2]:
import numpy as np
import pandas as pd
import math
import random
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LSTM,GRU,Dense, Dropout, Activation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
import sklearn.metrics as metrics
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# fix random seed for reproducibility
np.random.seed(1)
tf.random.set_seed(1) 
random.seed(1)

# read data from files to dataframes
df1 = pd.read_csv("./Traffic_Data_cali/train.csv", encoding='utf-8')
df2 = pd.read_csv("./Traffic_Data_cali/test.csv", encoding='utf-8')

# normalize data
scaler = MinMaxScaler(feature_range=(0, 1)).fit(df1['Flow (Veh/5 Minutes)'].values.reshape(-1, 1))
train_data = scaler.transform(df1['Flow (Veh/5 Minutes)'].values.reshape(-1, 1)).reshape(1, -1)[0]
test_data = scaler.transform(df2['Flow (Veh/5 Minutes)'].values.reshape(-1, 1)).reshape(1, -1)[0]

# practicing with different time lag (look back) values to optimize the models
lag = 12
train, test = [], []
for i in range(lag, len(train_data)):
    train.append(train_data[i - lag: i + 1])
for i in range(lag, len(test_data)):
    test.append(test_data[i - lag: i + 1])

train = np.array(train)
test = np.array(test)
# shuffle data (stateles case)
np.random.shuffle(train)
x_train = train[:, :-1]
y_train = train[:, -1]
x_test = test[:, :-1]
y_test = test[:, -1]

In [7]:
# building models

def build_LSTM():
    model = Sequential()
    model.add(LSTM(64,input_shape=(lag, 1),return_sequences=True))
    model.add(LSTM(64))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    return model    

def build_GRU():
    model = Sequential()
    model.add(GRU(64, input_shape=(lag, 1), return_sequences=True))
    model.add(GRU(64))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    return model

#model_struct = "LSTM"
model_struct = "GRU"
#model_struct = "AE"
# train the models 
if model_struct == "LSTM" :
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
    model = build_LSTM()
    model.compile(loss="mse", optimizer="rmsprop", metrics=['mape'])
    monitor = EarlyStopping(monitor='val_loss', patience=40, verbose=1, mode='auto',restore_best_weights=True)
    hist = model.fit(x_train, y_train,batch_size=64,epochs=600,callbacks=[monitor],validation_split=0.05)
    model.save('models/LSTM.h5')
    df = pd.DataFrame.from_dict(hist.history)
    df.to_csv('models/LSTM_loss.csv', encoding='utf-8', index=False)        
elif model_struct == "GRU" :        
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
    model = build_GRU()
    model.compile(loss="mse", optimizer="rmsprop", metrics=['mape'])
    monitor = EarlyStopping(monitor='val_loss', patience=40, verbose=1, mode='auto',restore_best_weights=True)
    hist = model.fit(x_train, y_train,batch_size=64,epochs=600,callbacks=[monitor],validation_split=0.05)
    model.save('models/GRU.h5')
    df = pd.DataFrame.from_dict(hist.history)
    df.to_csv('models/GRU_loss.csv', encoding='utf-8', index=False)


Epoch 1/600
116/116 [==============================] - 17s 147ms/step - loss: 0.0102 - mape: 95662.1953 - val_loss: 0.0040 - val_mape: 94222.0234
Epoch 2/600
116/116 [==============================] - 12s 103ms/step - loss: 0.0041 - mape: 30831.0547 - val_loss: 0.0057 - val_mape: 27574.7871
Epoch 3/600
116/116 [==============================] - 13s 111ms/step - loss: 0.0035 - mape: 21754.2148 - val_loss: 0.0045 - val_mape: 68690.5234
Epoch 4/600
116/116 [==============================] - 17s 150ms/step - loss: 0.0033 - mape: 19119.1699 - val_loss: 0.0035 - val_mape: 109682.6719
Epoch 5/600
116/116 [==============================] - 12s 103ms/step - loss: 0.0032 - mape: 23767.2305 - val_loss: 0.0030 - val_mape: 152016.2969
Epoch 6/600
116/116 [==============================] - 14s 123ms/step - loss: 0.0031 - mape: 28901.2305 - val_loss: 0.0031 - val_mape: 42638.7344
Epoch 7/600
116/116 [==============================] - 15s 130ms/step - loss: 0.0031 - mape: 26535.6230 - val_loss: 0.0027

Epoch 57/600
116/116 [==============================] - 12s 102ms/step - loss: 0.0025 - mape: 20281.8848 - val_loss: 0.0025 - val_mape: 35236.0508
Epoch 58/600
116/116 [==============================] - 18s 157ms/step - loss: 0.0026 - mape: 18731.8242 - val_loss: 0.0024 - val_mape: 69319.4219
Epoch 59/600
116/116 [==============================] - 15s 126ms/step - loss: 0.0026 - mape: 16437.9570 - val_loss: 0.0023 - val_mape: 59904.2891
Epoch 60/600
116/116 [==============================] - 13s 114ms/step - loss: 0.0025 - mape: 27275.6250 - val_loss: 0.0025 - val_mape: 69635.3438
Epoch 61/600
116/116 [==============================] - 18s 156ms/step - loss: 0.0026 - mape: 19459.7383 - val_loss: 0.0023 - val_mape: 56124.6797
Epoch 62/600
116/116 [==============================] - 15s 133ms/step - loss: 0.0026 - mape: 22163.2500 - val_loss: 0.0025 - val_mape: 68280.3594
Epoch 63/600
116/116 [==============================] - 13s 111ms/step - loss: 0.0025 - mape: 20794.3438 - val_loss: 0

116/116 [==============================] - 16s 137ms/step - loss: 0.0024 - mape: 23701.9961 - val_loss: 0.0022 - val_mape: 52709.4688
Epoch 113/600
116/116 [==============================] - 19s 165ms/step - loss: 0.0024 - mape: 21021.7656 - val_loss: 0.0021 - val_mape: 64354.1289
Epoch 114/600
116/116 [==============================] - 15s 127ms/step - loss: 0.0024 - mape: 17959.2383 - val_loss: 0.0023 - val_mape: 76418.3828
Epoch 115/600
116/116 [==============================] - 14s 120ms/step - loss: 0.0024 - mape: 16835.1582 - val_loss: 0.0023 - val_mape: 75385.2812
Epoch 116/600
116/116 [==============================] - 19s 164ms/step - loss: 0.0024 - mape: 23850.3574 - val_loss: 0.0021 - val_mape: 75853.0859
Epoch 117/600
116/116 [==============================] - 19s 161ms/step - loss: 0.0023 - mape: 18498.5586 - val_loss: 0.0023 - val_mape: 70293.2266
Epoch 118/600
116/116 [==============================] - 18s 159ms/step - loss: 0.0023 - mape: 14443.8965 - val_loss: 0.0022 -

In [13]:
# evaluate models and plot graphs 

# calculate Mean Absolute Percentage Error
def evaluate_models(y_true, y_pred):
    # the zero values we have in the dataset is a result to the sensors not reporting the data 
    # for that time for example server error or other types of errors we drop those values 
    # so they dont have false impact on the result 
    y_true = [x for x in y_true if x > 0]
    y_pred = [y_pred[i] for i in range(len(y_true)) if y_true[i] > 0]
      
    # calculate the Mean Absolute Percentage Error 
    sums = 0 # initialize value
    for i in range(len(y_pred)):
        tmp = abs(y_true[i] - y_pred[i]) / y_true[i]
        sums += tmp
    mape = sums * (100 / len(y_pred))
    # calculate variance score     
    vs = metrics.explained_variance_score(y_true, y_pred)
    mae = metrics.mean_absolute_error(y_true, y_pred)
    mse = metrics.mean_squared_error(y_true, y_pred)
    r2 = metrics.r2_score(y_true, y_pred)
    print('explained_variance_score:%f' % vs)
    print('mape:%f%%' % mape)
    print('mae:%f' % mae)
    print('mse:%f' % mse)
    print('rmse:%f' % math.sqrt(mse))
    print('r2:%f' % r2)
    
# def plot_LSTM_oneDay(y_true, y_pred):
#     d = '2020-3-4 00:00'
#     x = pd.date_range(d, periods=288, freq='5min')
#     fig = plt.figure(figsize=(20,10))
#     ax = fig.add_subplot(111)
#     ax.plot(x, y_true, label='True Data')
#     for name, y_pred in zip(['LSTM'], y_preds):
#         ax.plot(x, y_pred[: 288], label=name)
#     plt.legend()
#     plt.grid(True)
#     plt.xlabel('Time')
#     plt.ylabel('Flow')
#     date_format = mpl.dates.DateFormatter("%H:%M")
#     ax.xaxis.set_major_formatter(date_format)
#     fig.autofmt_xdate()
#     fig.savefig('./graphs/LSTM_oneDay.png')
#     plt.show()    
def plot_GRU_oneDay(y_true, y_pred):
    d = '2020-3-4 00:00'
    x = pd.date_range(d, periods=288, freq='5min')
    fig = plt.figure(figsize=(20,10))
    ax = fig.add_subplot(111)
    ax.plot(x, y_true, label='True Data')
    for name, y_pred in zip(['GRU'], y_preds):
        ax.plot(x, y_pred[: 288], label=name)
    plt.legend()
    plt.grid(True)
    plt.xlabel('Time')
    plt.ylabel('Flow')
    date_format = mpl.dates.DateFormatter("%H:%M")
    ax.xaxis.set_major_formatter(date_format)
    fig.autofmt_xdate()
    fig.savefig('./graphs/GRU_oneDay.png')
    plt.show()    
# def plot_models_oneDay(y_true, y_pred):
#     d = '2020-3-4 00:00'
#     x = pd.date_range(d, periods=288, freq='5min')
#     fig = plt.figure(figsize=(20,10))
#     ax = fig.add_subplot(111)
#     ax.plot(x, y_true, label='True Data')
#     for name, y_pred in zip(['LSTM', 'GRU'], y_preds):
#         ax.plot(x, y_pred[: 288], label=name)
#     plt.legend()
#     plt.grid(True)
#     plt.xlabel('Time')
#     plt.ylabel('Flow')
#     date_format = mpl.dates.DateFormatter("%H:%M")
#     ax.xaxis.set_major_formatter(date_format)
#     fig.autofmt_xdate()
#     fig.savefig('./graphs/models_oneDay.png')
#     plt.show()
# def plot_models_oneWeek(y_true, y_pred):
#     x = pd.date_range(start ='2020-3-4 00:00',end ='2020-3-11 00:00', freq ='5min')  
#     fig = plt.figure(figsize=(20,10))
#     ax = fig.add_subplot(111)
#     ax.plot(x, y_true, label='True Data')
#     for name, y_pred in zip(['LSTM', 'GRU'], y_preds):
#         ax.plot(x, y_pred[: 2017], label=name)
#     plt.legend()
#     plt.grid(True)
#     plt.xlabel('Time')
#     plt.ylabel('Flow')
#     date_format = mpl.dates.DateFormatter("%H:%M")
#     ax.xaxis.set_major_formatter(date_format)
#     fig.autofmt_xdate()
#     fig.savefig('./graphs/models_oneWeek.png')
#     plt.show()    

In [14]:
# lstm = load_model('./models/LSTM.h5')
gru = load_model('./models/GRU.h5')

In [15]:
models = [lstm, gru]
names = ['LSTM', 'GRU']
y_test = scaler.inverse_transform(y_test.reshape(-1, 1)).reshape(1, -1)[0]
y_preds = []
for name, model in zip(names, models):
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
    file = './images/' + name + '.png'
    plot_model(model, to_file=file, show_shapes=True)  
    predicted = model.predict(x_test)
    predicted = scaler.inverse_transform(predicted.reshape(-1, 1)).reshape(1, -1)[0]
    y_preds.append(predicted)
    print(name)
    evaluate_models(y_test, predicted)

NameError: name 'lstm' is not defined

In [ ]:
plot_LSTM_oneDay(y_test[: 288],y_preds[: 288])  

In [16]:
plot_GRU_oneDay(y_test[: 288],y_preds[: 288])  

NameError: name 'y_preds' is not defined

In [ ]:
plot_models_oneDay(y_test[: 288],y_preds[: 288])  

In [ ]:
plot_models_oneWeek(y_test[: 2017],y_preds[: 2017])  

In [ ]:
x=df2.loc[df2['5 Minutes'] =="11/3/2020 11:30"] 
true = x["Flow (Veh/5 Minutes)"].values
true[0]


In [ ]:
1578.shape

In [ ]:
predicted[1578].astype(int)